In [ ]:
import networkx as nx

In [ ]:
G = nx.read_graphml('G_2021.graphml')

In [ ]:
G.is_directed()

In [ ]:
# G = G.to_undirected()

In [ ]:
# G.is_directed()

In [ ]:
import pycombo
import networkx as nx

partition, modularity = pycombo.execute(
    G, weight="weight", return_modularity=True, random_seed=7575
)

In [ ]:
print(f'Partition modularity: {modularity:.5f}')

In [ ]:
G.nodes(data=True)

In [ ]:
from tqdm import tqdm

for node in tqdm(G.nodes()):
    G.nodes[node]['group'] = partition[node]

In [ ]:
nx.write_graphml(G, 'G_dists_clusters3.graphml')

In [ ]:
# --------------------------------------------------------------
# Дальше просто делаю где из точек и ребер. Не Комбо.

In [ ]:
import momepy as mp

In [ ]:
from shapely import Point
from tqdm import tqdm
from shapely.wkt import loads

for k,v in tqdm(dict(G.nodes(data=True)).items()):
    try:
        G.nodes[k]['x'], G.nodes[k]['y'] = loads(v['geometry']).coords.xy
        G.nodes[k]['x'] = float(G.nodes[k]['x'][0])
        G.nodes[k]['y'] = float(G.nodes[k]['y'][0])
        G.nodes[k]['is_valid_geom'] = True

    except IndexError:
        G.nodes[k]['is_valid_geom'] = False

    

In [ ]:
import geopandas as gpd
import networkx as nx

# assuming you already have a graph G with node data that includes a "geometry" attribute
# use the nodes method from networkx to get a NodeDataView object
nodedata = G.nodes(data=True)

# create a list of dictionaries to store the attributes of each node
c = 0
node_attrs = []
for n, data in nodedata:
    
    node_dict = data.copy()
    if 'geometry' in node_dict.keys():
        node_dict["nodeID"] = n
        node_dict["geometry"] = loads(node_dict["geometry"])
        node_attrs.append(node_dict)

        if n == 'Портовая особая экономическая зона':
            print(node_dict)
    else:
        c+=1
print(c, 'invaalid geoms')

# create a GeoDataFrame from the list of node attributes, using the "geometry" attribute as the geometry
nodes_gdf = gpd.GeoDataFrame(node_attrs, geometry='geometry', crs=4326)

In [ ]:
edgedata

In [ ]:
# nodes_gdf = nodes_gdf[nodes_gdf['is_valid_geom']==True]

In [ ]:
nodes_gdf.head()

In [ ]:
nodes_gdf.drop(columns='is_valid_geom', inplace=True)

In [ ]:
nodes_gdf.to_file('nodes_gdf_2021.geojson')

In [ ]:
nodes_gdf.crs

In [ ]:
import geopandas as gpd
import networkx as nx

import pandas as pd

# assuming you already have a graph G with node data that includes a "geometry" attribute
# use the nodes method from networkx to get a NodeDataView object
edgedata = G.edges(data=True)

# create a list of dictionaries to store the attributes of each node
c = 0
edge_attrs = []
for c1, c2, data in edgedata:
    
    edge_dict = {}
    
    edge_dict["city_1"] = c1
    edge_dict["city_2"] = c2
    edge_dict.update(data)
    edge_attrs.append(edge_dict)
    
# create a GeoDataFrame from the list of node attributes, using the "geometry" attribute as the geometry
edge_gdf = pd.DataFrame(edge_attrs)

In [ ]:
nodes_gdf.head()

In [ ]:
edge_gdf.iloc[753, :]

In [ ]:
from shapely.geometry import LineString
from tqdm import tqdm
tqdm.pandas()

def set_edge_geom(x):
    return LineString([nodes_gdf.loc[nodes_gdf["nodeID"]==x.city_1, "geometry"].item(), nodes_gdf.loc[nodes_gdf["nodeID"]==x.city_2, "geometry"].item()])

edge_gdf['geometry'] = edge_gdf.progress_apply(lambda x: set_edge_geom(x), axis=1)

In [ ]:
edge_gdf = gpd.GeoDataFrame(data=edge_gdf, geometry='geometry', crs=4326)

In [ ]:
edge_gdf.crs

In [ ]:
edge_gdf.to_file('edges_gdf_2021.geojson')